In [11]:
import django
import sys, os

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *

import pandas as pd
import numpy as np
import datetime

In [12]:
columns = [
    'id',
    'UT_id',
    'authors',
    'content',
    'first_author',
    'title',
    'wosarticle__de',
    'wosarticle__wc'
]

In [13]:
q = Query.objects.get(pk=1457)

doubletag = Tag.objects.get(pk=1405)

cutoff = datetime.datetime(2018,6,1,0,0,0)

rated_docs = DocOwnership.objects.filter(
    tag__query=1457,
    date__gt=cutoff,
    date__isnull=False
).exclude(tag=doubletag).values_list('doc__id',flat=True)

ratings = list(Doc.objects.filter(id__in=rated_docs,content__iregex='\w').values(*columns))
for r in ratings:
    d = Doc.objects.get(pk=r['id'])
    dos = DocOwnership.objects.filter(query=q,doc=d,relevant__gt=0)
    if dos.count() == 1:
        r['tag'] = dos.first().tag_id
        r['relevant'] = dos.first().relevant
        r['rated'] = dos.first().date
    elif dos.count() > 1:
        r['tag'] = dos.first().tag_id
        if dos.filter(relevant=1).count()>0:
            r['relevant'] = 1
            r['rated'] = dos.first().date
        else:
            r['relevant'] = dos.order_by('relevant').first().relevant
            r['rated'] = dos.first().date
    else:
        r['tag'] = ""
        r['relevant'] = 8
        r['rated'] = ""
        
df = pd.DataFrame.from_dict(ratings)

df = df[df['relevant'].isin([1,2])]

df['relevant'][df['relevant']==2] = 0

df = df.reset_index(drop=True)

df.head()


/home/galm/software/django/tmvenv_36/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1421: RuntimeWarning: DateTimeField DocOwnership.date received a naive datetime (2018-06-01 00:00:00) while time zone support is active.
  RuntimeWarning)


,UT_id,authors,content,first_author,id,rated,relevant,tag,title,wosarticle__de,wosarticle__wc
0,WOS:000368178300013,"Ben Said, A, Shahzad, MK, Zamai, E, Hubac, S, ...",Increasing demand diversity has resulted in hi...,"Ben Said, A",145433,2018-12-04 16:49:52.435801+00:00,0,1328,Experts' knowledge renewal and maintenance act...,Maintenance optimization; Dynamic knowledge ma...,"[Engineering, Industrial; Ergonomics]"
1,WOS:000239980800031,"Chan, APC, Chan, DWM, Fan, LCN, Lam, PTI, Yeun...",Project partnering has gained popularity in th...,"Chan, APC",88732,2018-09-21 02:32:31.206470+00:00,0,753,Partnering for construction excellence - A rea...,project partnering; partnering framework; comp...,[Construction & Building Technology; Engineeri...
2,WOS:000244233900004,"Kangas, A, Leskinen, P, Kangas, J",Criteria used to measure ecological or social ...,"Kangas, A",195165,2018-09-24 13:13:44.179815+00:00,1,760,Comparison of fuzzy and statistical approaches...,decisionmaking; uncertainty; preferences; risk,[Forestry]
3,WOS:000285823100009,"Kagan, S",Contemporary developments around the search fo...,"Kagan, S",229330,2018-09-21 12:42:54.004935+00:00,1,760,Cultures of sustainability and the aesthetics ...,None,[Economics; Planning & Development]
4,WOS:000337760300005,"Sato, S, Cuntz, M, Olvera, CMG, Jack, D, Schro...",We explore the general astrobiological signifi...,"Sato, S",685808,2018-12-04 16:49:52.493706+00:00,0,1328,Habitability around F-type stars,extrasolar planets; extraterrestrial life; F-t...,[Astronomy & Astrophysics; Biology; Geoscience...


In [14]:
df.shape

(2964, 11)

In [15]:
df.to_csv('docs.csv')